In [1]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

/home/nahid/anaconda3/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
import sys,os, pickle

import torch.nn as nn
from transformers import BertModel, BertTokenizer
from transformers import BertForMaskedLM
from torch.utils.data import DataLoader, Dataset

from transformers import AdamW
from tqdm import tqdm  # for our progress bar



with (open('DFGDATA.pkl' , "rb")) as openfile:
    all_input_list , all_label_list = pickle.load(openfile)


# input_text = ["I love apples.", "She hates oranges.", "I enjoy reading books.", "He likes playing football."]

# labels_task1 = [0, 1,0,1]  # Example task 1 labels
# labels_task2 = [2, 3,4,5]  # Example task 2 labels
BATCH_SIZE = 4

input_text = all_input_list#[:80]
labels_dfg = all_label_list#[:80]

labels_dfg =  torch.LongTensor(labels_dfg)

In [3]:
tokenizer  = BertTokenizer.from_pretrained("./binary-tokenizer")
VOCAB_SIZE = tokenizer.vocab_size

print('VOCAB SIZE : ', (tokenizer.vocab_size))


VOCAB SIZE :  7718


In [4]:
# from transformers import DataCollatorForLanguageModeling

# tokenizer.eos_token = "<EOI>"

# tokenizer.pad_token = tokenizer.eos_token
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [5]:
# data_collator(input_text_task1)

In [6]:
#TODO fix inputs = tokenizer(history, next_instruction, return_tensors='pt', 
#                    max_length=64, truncation=True, padding='max_length')

# Tokenize input text
inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors='pt',)
# encoded_inputs_task2 = tokenizer(input_text_task2, padding=True, truncation=True, return_tensors='pt')

inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [7]:
inputs['labels_dfg'] = labels_dfg

In [8]:
#MLM
inputs['labels'] = inputs.input_ids.detach().clone()
inputs.labels[0]

# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
        (inputs.input_ids != 102) * (inputs.input_ids != 0)

#TODO FIX 101, 102

# mask_arr[0]

In [9]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )
# selection[0]

In [10]:
# inputs.input_ids

In [11]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels_dfg', 'labels'])

In [12]:
print(inputs.keys())
print(inputs.token_type_ids)
inputs.token_type_ids[0][0] =1
print(inputs.token_type_ids)
#TODO set token type ids

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels_dfg', 'labels'])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


In [13]:
class BinaryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [14]:
dataset = BinaryDataset(inputs)
train_size = int(0.8 * len(dataset))
validation_size = len(dataset) - train_size

train_dataset, validation_dataset = torch.utils.data.random_split(dataset, [train_size, validation_size])
train_loader      = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [15]:

# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Define the multitask BERT model

hidden_size = BertModel.from_pretrained('bert-base-uncased').config.hidden_size
# vocab_size = bert_model.config.vocab_size
class MultitaskBert(nn.Module):
    def __init__(self):
        super(MultitaskBert, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased') #BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(  hidden_size , VOCAB_SIZE)
        self.dfg_classifier = nn.Linear(hidden_size, 2) #todo was 768 num_classes_task2



    def forward(self, input_ids, attention_mask ):
        bert_outputs = self.bert(input_ids, attention_mask=attention_mask)

        hidden_states = bert_outputs.last_hidden_state
        logits_mlm = self.fc(hidden_states)
        
        bert_pooled_output = bert_outputs.pooler_output
        logits_dfg = self.dfg_classifier(bert_pooled_output)
        
        
        return logits_mlm,   logits_dfg




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
# Create model instance
model = MultitaskBert()  # Example number of classes for each task
model.to(device)

# initialize optimizer
optim = AdamW( model.parameters() , lr=5e-6)
criterion = nn.CrossEntropyLoss()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
EPOCHS = 10


global_instruction_metrices = []
global_token_metrices = []

v_global_instruction_metrices = []
v_global_token_metrices = []

for ecpoch in range(EPOCHS):
    train_loop = tqdm(train_loader, leave=True)
    
    
    model.train()
    for N,batch in enumerate(train_loop):


        # Forward pass
        optim.zero_grad()
        # logits_task1, logits_task2 , t1_outputs,t2_outputs = model(inputs['input_ids'], inputs['attention_mask'], inputs['input_ids'], inputs['attention_mask'])

        logits_task1,   logits_task2 = model(batch['input_ids'].to(device), batch['attention_mask'].to(device))
#         print(logits_task1.shape, logits_task2.shape)
#         print(logits_task1.shape , batch['labels'].shape)
#         logits_task1.view(-1, VOCAB_SIZE).shape ,batch['labels'].view(-1).shape
        
        loss_task1 = criterion(logits_task1.view(-1, VOCAB_SIZE), batch['labels'].to(device).view(-1))
        predictions_task1 = torch.argmax(logits_task1, dim=2)
        

        
        
        loss_task2 = criterion(logits_task2, batch['labels_dfg'].to(device))
        predictions_task2 = torch.argmax(logits_task2, dim=1)
        
        (loss_task1+loss_task2).backward()
        optim.step()
        
        
        # print relevant info to progress bar
        train_loop.set_description(f'Epoch {ecpoch}')
        train_loop.set_postfix(loss=(loss_task1+loss_task2).item())
        
        
        

  0%|                                                  | 0/7706 [00:00<?, ?it/s]/tmp/ipykernel_51065/3277124069.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 1:  59%|███████████▊        | 4545/7706 [39:03<27:14,  1.93it/s, loss=1.1]

In [ ]:
print('done')